# Pfeile

## Aufgaben

- Visualisieren mit interaktiver Grafik
- Statistische Kennzahlen berechnen
- Berechnen der Fehlerellipse
- Standartabweichung in Winkelstreuung umwandeln

## Arbeitsplan
1. Daten Aufbereiten
2. Alle Nötigen Daten berechnen
    - Mittelwerte
    - Standartabweichung
    - Streuung
    - Winkelstreuung
    - Fehlerellipse
3. Erste Visualisierungen
4. Erstellen der Interaktiven Grafik
5. Optimierung

# Wie kommen die Daten im Bogensport zustande?

## Distanz

Die Distanz der Sportler zur Zielscheibe variiert je nach Alter, größe der Zielscheibe und des Wettbewerbs. Der ASC Göttingen zeigt auf Ihrer Website über das Olympische Bogenschießen - welches nach den Regeln der World Archery stattfindet - die verschiedenen Distanzen und Altersgruppen der Sportler. Zum Beispiel schießen in dem Wettbewerb 13 - 14 jährige aus 40 Meter Distanz auf eine Zielscheibe mit 80cm Durchmesser $ ^3$.

## Zielscheiben

Zielscheiben müssen fest an dem Boden befästigt werden um eine stabile Zielscheibe gewährleisten zu können. Außerdem müssen Bereiche der Zielscheibenhalterung abgedeckt werden, falls diese die Pfeile beschädigen könnten $ ^1$. 

Es gibt zehn verschiedene Zielscheiben für den Bogensport in geschlossenen Räumen (60cm oder 40cm durchmesser) sowie vier für das Bogenschießen im Freien (122cm oder 80cm). Diese Zielscheiben unterscheiden sich nicht nur in der Größe sondern auch die Anzahl der Zielscheiben und wie sie geordnet sind. Zum Beispiel besteht das "The 60cm vertical triple face" aus drei Zielscheiben mit 60cm Durchmesser die vertikal angeordnet sind $ ^1$.

Der Mittelpunkt der Zielscheibe muss 130cm über dem Boden sein. Es gibt eine Toleranz von 5cm $ ^1$.

## Passe

Eine Passe ist gleichzusetzen mit dem Wort "Durchlauf". Ein Durchlauf beginnt wenn der Spieler nun am Zug ist und ist dann beendet wenn der Sportler eine gewisse Anzahl an Pfeilen abgeschossen hat und nun der nächste Spieler am Zug ist oder die gesamte Runde vorbei ist.

## Wertung der Punkte

Jeder abgeschossene Pfeil muss in der Zielscheibe fest hängenbleiben, andernfalls bleibt die Wertung der getroffenen Punktzahl aus. Jede Zielscheibe besteht aus zehn Ringen wobei der Zehnte sich in der Mitte und der erste am äußersten befindet. Zudem gibt es in dem 10 Ring noch den "elften Ring" namens innere zehn welcher die höchsmögliche Punktzahl der Zielscheibe widerspiegelt $ ^1$. Die Wertung der Punkte erfolgt nach jeder Passe und wird auf einer Punktekarte absteigend festgehalten $ ^2$.

In [ ]:
import pandas as pd
from sage.plot.scatter_plot import ScatterPlot
import numpy as np
from sage.plot.ellipse import Ellipse
from ipywidgets import interact
import ipywidgets as widgets
from sage.plot.circle import circle
import pathlib

### Warnings deaktivieren da sie Teilweise die Interaktive Grafik Stören
Bei Problemen diese Zelle nicht ausführen

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Klasse "Pfeildaten"

Enthält funktionen um Informationen aus Gespeicherten Dataframes zu lesen

In [ ]:
class Pfeildaten():
    def __init__(self, pfad):
        '''Nimmt den Namen eins Dateipfades aus dem momentanen Directory
        und speichert alle .csv Dateien aus diesem Ordner in einem Dictionary
        mit den Dateinamen als Keys'''
        self.data = {file.name: pd.read_csv(file, delimiter=";") for file in pathlib.Path("./" + pfad).glob('*.csv')}
    
    @staticmethod
    def calcMid(data):
        '''Gibt ein tupel mit den mitleren x und y Werten'''
        summe_x = sum(data["x"])
        summe_y = sum(data["y"])
        count = data.shape[0]
        return (summe_x/count, summe_y/count)

    @staticmethod
    def Streuung(data):
        '''Gibt ein tupel mit Standartabeichung in x und y Richtung wieder.'''
        std_x = np.std(data["x"])
        std_y = np.std(data["y"])
        return (std_x, std_y)

    
    def WinkelStreuung(self, data):
        '''Gibt ein tupel mit mit Winkelstreuung in x und y Richtung wieder.'''
        a = list(set(data["Distanz"]))
        return (self.Streuung(data)[0]/a[0], self.Streuung(data)[1]/a[0])

    
    def combine(self, keys):
        '''Nimmt eine Liste von Keys und kombiniert die jeweiligen dataframes aus self.data zu einem dataframe'''
        return pd.concat([self.data[a] for a in keys], axis=0)

    
    def datafromwidgets(self, selected_Data, range, ringe, passe):
        '''Filtert die ausgewählten Dataframes nach Distanz, Ringenzahl und Passenzahl
        mithilfe der Methoden filter_Distanz, filter_passe, filter_ringe.
        Gibt das gefilterte Dataframe zurück.'''
        if len(selected_Data) == 0: pass
        else:
            return self.filter_ringe(self.filter_passe(self.filter_Distanz(self.combine(selected_Data), range), passe), ringe)
            
    
    def getDataFromRange(self, range):
        '''Nimmt einen Reichweite Parameter und gibt alle Namen von Dataframes zurück die diese Reichweite enthalten'''
        return [key for key, value in list(self.data.items()) if range in list(value["Distanz"])]

    
    def getKeys(self):
        '''gibt eine Liste aus allen Keys von self.data zurück'''
        return list(self.data.keys())
    
    
    def getMaxRing(self, data):
        '''Gibt den Maximalen Wert für den getroffenen Ring aus einem DataFrame'''
        return max(list(self.combine(data)["Ringe"]))
        
    
    def getMaxPasse(self, data):
        '''Gibt den Maximalen Wert der Splate Passe aus ausgewählten Dataframes zurück'''
        return max(list(self.combine(data)["Passe"]))

    
    def getMaxSize(self, data):
        '''gibt den größten Wert aus der Spalte "Typ Auflage" aus ausgewählten Dataframes zurück'''
        return max(list(self.combine(data)["Typ Auflage"]))
        
    
    def getRanges(self):
        '''gibt eine Liste an allen Distanzwerten aus self.data'''
        return list(set(list(self.combine(self.getKeys())["Distanz"])))


    @staticmethod
    def filter_ringe(datensatz, ringe):
        '''Gibt ein Dataframe zurück das nach angegebener Ringzahl gefiltert wurde.
        Nimmt ein Dataframe und Ringzahl.'''
        if ringe == 0:
            return datensatz
        else: 
            return datensatz[datensatz["Ringe"] == ringe]
    

    @staticmethod
    def filter_passe(datensatz, passe):
        '''Gibt ein Dataframe zurück das nach angegebener Passenzahl gefiltert wurde.
        Nimmt ein Dataframe und Passenzahl.'''
        if passe == 0:
            return datensatz
        else: 
            return datensatz[datensatz["Passe"] == passe]
    
    @staticmethod
    def filter_Distanz(datensatz, Distanz):
        '''filtert ein Dataframe nach dem Parameter Distanz'''
        return datensatz[datensatz["Distanz"] == Distanz]

    @staticmethod
    def Punkte(datensatz):
        '''Nimmt einen gefilterten Datensatz (nach Ringe und Passe) 
        und gibt eine Liste mit Tupeln (zu plottende Punkte) zurück.'''
        filtered_data = datensatz
        return [(list(filtered_data["x"])[a], list(filtered_data["y"])[a]) for a in range(len(filtered_data))]
    


## Klasse "Plots"

1. Erbt die Funktionen der Klasse "Pfeildaten"
2. Enthält Funktionen zum Plotten der Daten

In [ ]:
class Plots(Pfeildaten):
    def __init__(self, pfad):
        super().__init__(pfad)
    
    def drawMid(self, data):
        '''Zeichnet ein Kreuz an den Mittelpunkt der Daten'''
        return scatter_plot([self.calcMid(data)], markersize=100, marker="+", facecolor="red")

    def drawErrorElipse(self, data):
        '''Zeichnet die Fehlerellipse der Angegebenen Daten'''
        
        # Eigenwerte und Verktoren der Kovarianzmatrix aus den x- und y- Koordinaten berechnen
        Eigenvalues = matrix(np.cov(data["x"], data["y"])).eigenvectors_right()

        # Den Drehwinkel der Elipse aus der Ausrichtung des Eigenvektors zum Größten Eigenwert bestimmen
        angle = arctan(Eigenvalues[0][1][0][1]/Eigenvalues[0][1][0][0])

        # 5.991 ist ein Standartwert sodass die Elipse 95% der Daten beinhaltet
        return ellipse(
            self.calcMid(data),
            sqrt(5.991 * Eigenvalues[0][0]),
            sqrt(5.991 * Eigenvalues[1][0]),
            angle,
            edgecolor="green",
        )


    def drawStandardDeviation(self, data):
        x, y = var("x, y")
        
        min_x = min(data["x"])
        max_x = max(data["x"])
        min_y = min(data["y"])
        max_y = max(data["y"]) 

        # Berechne Mittelwerte und Standardabweichungen
        mid_x = self.calcMid(data)[0]
        mid_y = self.calcMid(data)[1]
        std_x = self.Streuung(data)[0] 
        std_y = self.Streuung(data)[1]
        
        # Plotte Glockenkurve in horizontale Richtung
        bell_curve_x = plot(
            (1/(std_x*sqrt(2*pi)))*exp(-0.5*((x - mid_x)/std_x)**2),
            (x, min_x, max_x),
            color='red',
            legend_label="Horizontal",
            title="Standardabweichung",
            frame=True,
        )

        # Plotte Glockenkurve in vertikale Richtung
        bell_curve_y = plot(
            (1/(std_y*np.sqrt(2*np.pi)))*np.exp(-0.5*((y - mid_y)/std_y)**2),
            (y, min_y, max_y), color='blue',
            legend_label="Vertikal",
        )
        
        combined_plot = bell_curve_x + bell_curve_y
        print(f"Standardabweichung in horizontale Richtung: {round(std_x, 3)}\nStandardabweichung in vertikale Richtung: {round(std_y, 3)}")
        combined_plot.set_legend_options(loc="upper right")
        combined_plot.axes_labels(["X bzw Y Koordinate der Zielscheibe", "Wahrscheinlichkeitsdichte"])
        combined_plot.axes_labels_size(0.8)
        # Gibt kombinierte Grafik aus horizontaler und vertikaler Standardabweichung
        return combined_plot


    def Interaktiv(self, selected_Data, range, ringe, passe):
        '''Nimmt die durch den Benutzer ausgewählten Dataframes, Ringenzahl und Passenzahl.
           Gibt eine Interaktive Grafik der Zielscheibe inklusive Mittelpunkt der Punkte, 
           Fehlerellipse, Standardabweichung und Punkte zurück.'''
        if len(selected_Data) == 0: pass
        else:
            data = self.datafromwidgets(selected_Data, range, ringe, passe)       
            plot1 = list_plot(
                self.Punkte(data),
                color="blue",
                pointsize=10,
                figsize=6,
            )
            plot1 += circle(
                (0,0),
                self.getMaxSize(selected_Data)*5,
                edgecolor="black",
            )
            if len(data) >= 3: plot1 += self.drawErrorElipse(data)
            if len(data) > 1: plot1 += self.drawMid(data)
            if len(data) >= 3: 
                plot1.show()
                self.drawStandardDeviation(data).show()
            else: plot1.show()

### Instanz der Klasse erstellen

In [ ]:
Instanz1 = Plots("Pfeildaten")

### Winkelstreuung der Datensätze

In [ ]:
for key in Instanz1.data.keys():
    x = Instanz1.WinkelStreuung(Instanz1.data[key])[0]
    y = Instanz1.WinkelStreuung(Instanz1.data[key])[1]
    print(f"{key} Winkelstreuung:\nx-Richtung: {x}\ny-Richtung: {y}\n")

### Test der Fehlerellipse

- Drehen und Normieren der Datenpunkte damit wir die Formel einer normalen Ellipse nutzen können <br>
$$(\dfrac{x}{a})^2 + (\dfrac{y}{b})^2 = 1$$
- a, b sind die jeweiligen Radii der Ellipse <br>
- Ein Punkte ($x_i, y_i$) liegt innerhalb der Ellipse wenn $(\dfrac{x_i}{a})^2 + (\dfrac{y_i}{b})^2 < 1$


In [ ]:
def datenAnpassen(dataframe, Winkel):
    #Die Datenpunkte drehen und Normieren sodass der Mittelpunkt bei (0, 0) und die Ausrichtung der Fehlerellipse parallel zu x-Achse ist
    Punkte = list(zip(list(dataframe["x"] - Pfeildaten.calcMid(dataframe)[0]), list(dataframe["y"] - Pfeildaten.calcMid(dataframe)[1])))
    Matrix = np.matrix(Punkte)
    Gedreht = np.matmul(Matrix, [[cos(Winkel), -sin(Winkel)], [sin(Winkel), cos(Winkel)]]).tolist()
    return Gedreht


def isinEllipse(dataframe):
    OriginalEigenvalues = matrix(np.cov(dataframe["x"], dataframe["y"])).eigenvectors_right()
    angle = arctan(OriginalEigenvalues[0][1][0][1]/OriginalEigenvalues[0][1][0][0])
    newData = datenAnpassen(dataframe, angle)
    newEVs = matrix(np.cov([x for x, y in newData], [y for x, y in newData])).eigenvectors_right()
    forget()
    x, y = var("x y")
    f = (x^2)/sqrt(5.991 * newEVs[0][0])^2 + (y^2)/sqrt(5.991 * newEVs[1][0])^2
    bInEllipse = [True if float(f.substitute(x=xValue, y=yValue)) < 1 else False for xValue, yValue in newData] 
    return int(bInEllipse.count(true)/len(bInEllipse)*100)

for key in Instanz1.data.keys():
    print(f"Die Errorellipse zu Datensatz {key} umschließt {isinEllipse(Instanz1.data[key])}% der Datenpunkte")

Diese Werte liegen alle in der Nähe des Wertes von 95% welcher durch den Faktor 5.991 zu erwarten ist


### Prüfen ob der Plot der Warscheinlichkeitsdichte Plausibel ist
- Erster Test: Ist der Hochpunkt an der Richtigen Stelle
- Zweiter Test: Ist die Fläche unter der Kurve = 1

In [ ]:
def Standardabweichung_Test_Mittelpunkt(mid, std, min_x, max_x):
    x = var("x")
    f(x) = 1/(std*sqrt(2*pi))*exp(-0.5*((x - mid)/std)**2)
    if find_root(f.diff(x) == 0,min_x, max_x) == mid:
        print(True)
    else: print(False)                
    

def Standardabweichung_Test_Fläche(mid, std):
    f(x) = 1/(std*sqrt(2*pi))*exp(-0.5*((x - mid)/std)**2)
    if round(integral(f(x), x, -oo, +oo).n(), 15) == 1: # M
        mf = abs(integral(f(x), x, -oo, +oo).n()-1)
        print(f"True, Maschinenfehler beträgt: {mf} und ist somit zu vernachlässigen")
    else: print(False)   
    
Standardabweichung_Test_Mittelpunkt(-19.416, 42.075, -137, 132) # Test für arrows3 mit allen Datenpunkten
Standardabweichung_Test_Fläche(-19.416, 42.075) # Test für arrows3 mit allen Datenpunkten

## Interaktive Grafik

In [ ]:
Data_Widget = widgets.SelectMultiple(
    options = sorted(Instanz1.getDataFromRange(list(Instanz1.getRanges())[0])),
    value = [Instanz1.getDataFromRange(list(Instanz1.getRanges())[0])[0]],
    description="Daten",
    disabled = False
)
Range_Widget = widgets.RadioButtons(
    options=Instanz1.getRanges(),
    value=list(Instanz1.getRanges())[0],
    description="Distanz",
)
Ringe_Widget = widgets.FloatSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description="Ring",
)
Passe_Widget = widgets.FloatSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description="Passe",
)

def changeData_Widget(change):
    '''Wechselt die Auswahlmöglichkeiten an datensätzen basierend auf der auswahl der Distanz'''
    options = sorted(Instanz1.getDataFromRange(change.new))
    Data_Widget.options = options
    Data_Widget.value = [options[0]]

def changeRingePasse_WidgetMax(change):
    '''passt die Ring und Passe Slider an sodass das Maximum dem Datensatz entspricht'''
    if len(change.new) == 0: pass
    else:
        Ringe_Widget.max = Instanz1.getMaxRing(change.new)
        Passe_Widget.max = Instanz1.getMaxPasse(change.new)

Range_Widget.observe(changeData_Widget, names="value")
Data_Widget.observe(changeRingePasse_WidgetMax, names="value")

interact(
    Instanz1.Interaktiv,
    selected_Data=Data_Widget,
    range=Range_Widget,
    ringe=Ringe_Widget,
    passe=Passe_Widget,
)
None

# Arbeitsteilung


## Amadeus Harms:
### Berechnungsfunktionen:
- Mittelpunkt
- Standartabweichung
- Winkelstreuung
### Plotmethoden:
- Mittelpunkt
- Fehlerellipse
- Klassenstruktur
### Klassenstruktur
- Erstellen vom Klassen
- Einfügen und anpassen der Funktionen an die Klassen
- get- Funktionen implementiert
### Interaktive Grafik
- Change und Observe Funktionen zur Interaktiven Grafik hinzugefügt
### Tests
- Test der Fehlerellipse

## Enes Yasaroglu:
- Datenerhebung im Bogensport
- Recherche / Literaturverzeichnis
- Datenaufbereitung
### Methoden:
- Datensatz nach Ringen und Passen filtern
- Plotmethode der Standardabweichung
- Interaktive Grafik erstellt
- Punkte
### Tests
- Ist der Plot der Warscheinlichkeitsdichte plausibel

## Literaturverzeichnis
- [1] World Archery. (2024). Chapter 7 FIELD OF PLAY SETUP - TARGET ROUNDS. In Rulebook. Abgerufen am 28. April 2024, von https://www.worldarchery.sport/rulebook/article/13
- [2] World Archery. (2024). Chapter 14 SCORING. In Rulebook. Abgerufen am 28. April 2024, von https://www.worldarchery.sport/rulebook/article/903
- [3] Bogensport Göttingen. (n.d.). Olympisches Bogenschießen. Abgerufen am 28. April 2024, von https://www.bogenschiessen-goettingen.de/bogensport-olympisches-bogenschiessen.cfm

### Fehlerellipse Recherche (Amadeus):
- Vincent Spruyt. (2014). How to draw an error ellipse representing the covariance matrix?.[Online] visiondummy, 03.04.2014 [Abgerufen am 20.04.2024] https://www.visiondummy.com/2014/04/draw-error-ellipse-representing-covariance-matrix/

### Plot Standardabweichung (Enes):
 - Wikipedia. (2024). Normalverteilung. Abgerufen am 29.04.2024, von https://wikipedia.org/wiki/Normalverteilung